Parâmetros

In [0]:
%run /Users/wandrieli.89@gmail.com/projeto_tiller/Parametros/nb_parametros

In [0]:
%run /Users/wandrieli.89@gmail.com/projeto_tiller/Parametros/nb_catalag_schema

Regras e Definições

In [0]:
%sql
select * from tiller.silver.tb_despesas_silver;

In [0]:
set_catalog_and_schema_gold() #Define catalog e schema conforme run

In [0]:
query = f"""
SELECT
    ideDocumento AS id_despesas,
    txtDescricao AS tipo_despesa
FROM tiller.silver.tb_despesas_silver
"""

df = spark.sql(query)


df_despesas = (
    df
        .groupBy("id_despesas")
    .agg(
        F.first("tipo_despesa", ignorenulls=True).alias("tipo_despesa") # Precisei agrupar e pegar a primeira despesa para o id do agrupamento
    )

    .withColumn("data_ingestao", F.date_trunc("second", F.current_timestamp())) #date trunc para considerar somente até os segundos. 
    .withColumn("data_modificacao", F.date_trunc("second", F.current_timestamp()))
)

df_despesas.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("tiller.gold.dim_despesas")


Incluindo comentários nas colunas

In [0]:
%sql
COMMENT ON COLUMN tiller.gold.dim_despesas.id_despesas IS 'ID da despesa conforme o sistema fonte (ideDocumento)';
COMMENT ON COLUMN tiller.gold.dim_despesas.tipo_despesa IS 'Tipo de despesa, valor padronizado a partir do campo txtDescricao';
COMMENT ON COLUMN tiller.gold.dim_despesas.data_ingestao IS 'Data e hora da ingestão dos dados na tabela Gold';
COMMENT ON COLUMN tiller.gold.dim_despesas.data_modificacao IS 'Data e hora da última modificação registrada na tabela Gold';


In [0]:
%sql

SELECT COUNT(*) AS total, count(distinct id_despesas) as ids, MIN(data_ingestao) AS primeira_carga
FROM dim_despesas;


In [0]:
%sql
select * from tiller.gold.dim_despesas;